### Unsupervised

In [3]:
# #write config file 
# import json
# json_path ='C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/'
# # path = json_path + "VICE_JSON/"
# path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/"
# config = {   
#     "Email":"abc@intel.com", #for logging purpose
#     "user_outpath" :"C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/user_outpath/", 
#     "log_path" : "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/user_outpath/",
#     "DataLoading": {"path": path, "start_date": None,"stop_date":None},
#     "DataPreprocessing":{    
#     "df_manipulation": {"how":None,"col_selection":["id","description"],"keep": None,"subset":None},    
#     "target":{"enable":False,"column":"problem_area"}, #for supervised learning only
#     "word_contractions": {"enable": True},
#     "lowercase": {"enable": True},
#     "remove_htmltag_url": {"enable":True},
#     "remove_irrchar_punc":{"enable":True,"char":None},
#     "remove_num":{"enable":True},
#     "remove_multwhitespace":{"enable":True},
#     "remove_stopwords":{"enable":True,"extra_sw":None,"remove_sw": None},
#     "remove_freqwords":{"enable":True,"n":10},
#     "remove_rarewords":{"enable":True,"n":10},
#     "custom_taxo":{"enable":True,"remove_taxo":r'test \w+',"include_taxo":["test suite execution"]},
#     "stem_words":{"enable":True,"stemmer_type":None},
#     "lemmatize_words":{"enable":False,"lemma_type":None}
#     },
#    "UnsupervisedLearning":{
#         "Output": {"User":False,"ELK":True},
#         "elk_outpath" : "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/elk_outpath/",
#         "kmeans_clustering":{"enable":True,"top_n_terms":10,"ngram_range":None,"fe_type":None,"n_clusters":None,"max_n_clusters":5},
#         "lda":{"enable":True,"n_components":5,"top_n_terms":10,"ngram_range":None},
#         "nmf":{"enable":True,"n_components":5,"top_n_terms":10,"ngram_range":None,"fe_type":None}
#     },   
#     "SupervisedLearning":{
#         "supervised_lng":{"enable":False,"target":"problem_area","test_size":0.3,"ngram_range":None,"fe_type":None,"model_type":None,"ascend":None},
#         "deep_lng":{"enable":False,"target":"problem_area","test_size":0.3,"ngram_range":None,"fe_type":None,"hidden_layer_sizes":(5),"activation":None,"solver":None,"learning_rate":None,"max_iter":None,"ascend":None}
#     },
#     "SimilarityMetrics":{
#         "cosinesimilarity":{"enable":True,"threshold":0,"total_rows":10,"base_row":None,"ngram_range":None,"fe_type":None,"ascending":None},
#         "jaccardsimilarity":{"enable":True,"threshold":0,"total_rows":10,"base_row":None,"ascending":None}
#     }

# }

# with open(json_path+'config.json', 'w') as f:
#     json.dump(config,f,indent=8)

In [1]:
#write config file 
import json
json_path ='C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/'
# path = json_path + "VICE_JSON/"
path = "/nfs/png/home/nchong/Data/"
config = {   
    "Email":"abc@intel.com", #for logging purpose
    "user_outpath" :"/nfs/png/home/nchong/", 
    "log_path" : "/nfs/png/home/nchong/",
    "DataLoading": {"path": path, "start_date": None,"stop_date":None},
    "DataPreprocessing":{    
    "df_manipulation": {"how":None,"col_selection":["id","description"],"keep": None,"subset":None},    
    "target":{"enable":False,"column":"problem_area"}, #for supervised learning only
    "word_contractions": {"enable": True},
    "lowercase": {"enable": True},
    "remove_htmltag_url": {"enable":True},
    "remove_irrchar_punc":{"enable":True,"char":None},
    "remove_num":{"enable":True},
    "remove_multwhitespace":{"enable":True},
    "remove_stopwords":{"enable":True,"extra_sw":None,"remove_sw": None},
    "remove_freqwords":{"enable":True,"n":10},
    "remove_rarewords":{"enable":True,"n":10},
    "custom_taxo":{"enable":True,"remove_taxo":r'test \w+',"include_taxo":["test suite execution"]},
    "stem_words":{"enable":True,"stemmer_type":None},
    "lemmatize_words":{"enable":False,"lemma_type":None}
    },
   "UnsupervisedLearning":{
        "Output": {"User":False,"ELK":True},
        "elk_outpath" : "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/elk_outpath/",
        "kmeans_clustering":{"enable":True,"top_n_terms":10,"ngram_range":None,"fe_type":None,"n_clusters":None,"max_n_clusters":5},
        "lda":{"enable":True,"n_components":5,"top_n_terms":10,"ngram_range":None},
        "nmf":{"enable":True,"n_components":5,"top_n_terms":10,"ngram_range":None,"fe_type":None}
    },   
    "SupervisedLearning":{
        "supervised_lng":{"enable":False,"target":"problem_area","test_size":0.3,"ngram_range":None,"fe_type":None,"model_type":None,"ascend":None},
        "deep_lng":{"enable":False,"target":"problem_area","test_size":0.3,"ngram_range":None,"fe_type":None,"hidden_layer_sizes":(5),"activation":None,"solver":None,"learning_rate":None,"max_iter":None,"ascend":None}
    },
    "SimilarityMetrics":{
        "cosinesimilarity":{"enable":True,"threshold":0,"total_rows":10,"base_row":None,"ngram_range":None,"fe_type":None,"ascending":None},
        "jaccardsimilarity":{"enable":True,"threshold":0,"total_rows":10,"base_row":None,"ascending":None}
    }

}

with open(json_path+'config.json', 'w') as f:
    json.dump(config,f,indent=8)

In [ ]:
/nfs/png/home/nchong/

In [25]:
json_path ='C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/'
main()

Files read: ['C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/VICE_JSON/(2021-08-25)1_VICE_1.json']
Shape of df before manipulation: (2078, 20)
Shape of df after selecting columns: (2078, 3)
Number of null values in df:
 id              0
description     0
problem_area    0
dtype: int64
NA is imputed with empty string
Number of null values in df after NA imputation:
 id              0
description     0
problem_area    0
dtype: int64
Number of duplicates in the df: 0
Shape of df after manipulation: (2078, 3)


,id,description,problem_area
0,22062762,When executing the Get Port Bandwidth command ...,
1,220152061,"<pre style=""word-wrap: break-word; white-space...",
2,220421258,"<pre style=""word-wrap: break-word; white-space...",noise.cannot_reproduce
3,220634430,"<p style=""font-size: 12.18px;""><span style=""fo...",noise.cannot_reproduce
4,220634437,<p>Trying Recipe sent by Tamir</p><p>pci_confi...,noise.non-issue
...,...,...,...
2073,14010346691,"<div style=""direction:ltr"">\n\n<table border=""...",
2074,14010350584,<p>FPGA Image:&nbsp; 4_fpga_usbxcoe_top_tgph1p...,
2075,14010378525,<p>FPGA Image:</p><p><br /></p><p>This test is...,
2076,14010414934,<p>SVOS Kernel (BUSTER) :&nbsp;4.19.60<br /></...,noise.validation_tools


,description
0,When executing the Get Port Bandwidth command ...
1,"<pre style=""word-wrap: break-word; white-space..."
2,"<pre style=""word-wrap: break-word; white-space..."
3,"<p style=""font-size: 12.18px;""><span style=""fo..."
4,<p>Trying Recipe sent by Tamir</p><p>pci_confi...
...,...
2073,"<div style=""direction:ltr"">\n\n<table border=""..."
2074,<p>FPGA Image:&nbsp; 4_fpga_usbxcoe_top_tgph1p...
2075,<p>FPGA Image:</p><p><br /></p><p>This test is...
2076,<p>SVOS Kernel (BUSTER) :&nbsp;4.19.60<br /></...


,problem_area
0,
1,
2,noise.cannot_reproduce
3,noise.cannot_reproduce
4,noise.non-issue
...,...
2073,
2074,
2075,
2076,noise.validation_tools


Frequent words that are removed: {('u', 6887), ('b', 7516), ('h', 6339), ('p', 7860), ('c', 6867), ('v', 6994), ('r', 6319), ('f', 7379), ('x', 12523), ('n', 6332)}
Rare words that are removed: {('conform', 1), ('narrowing', 1), ('environmental', 1), ('reimaged', 1), ('tnolfpsresponsetimeout', 1), ('responsive', 1), ('resolving', 1), ('involved', 1), ('sbrt', 1), ('elapses', 1)}


,problem_area,description_cont_lower_tagrem_puncrem_numrem_wsrem_stoprem_freqrem_rarerem_taxo_stem
0,,execut get port bandwidth command msle ssp tra...
1,,current see dpm issu unit usb port usb port mi...
2,noise.cannot_reproduce,basin fall use server deriv cpu kbp pch design...
3,noise.cannot_reproduce,xhci debug devic fail cnl platform fail seed c...
4,noise.non-issue,tri recip sent tamir pci config regist access ...
...,...,...
2073,,hardwar asu prime z hap snp daughter card sier...
2074,,fpga imag fpga usbxco top tgph visa ww e e upd...
2075,,fpga imag intermitt pass fail suspect issu des...
2076,noise.validation_tools,svo kernel buster svf version svf modul svo de...


,problem_area,id,description,description_cont,description_cont_lower,description_cont_lower_tagrem,description_cont_lower_tagrem_puncrem,description_cont_lower_tagrem_puncrem_numrem,description_cont_lower_tagrem_puncrem_numrem_wsrem,description_cont_lower_tagrem_puncrem_numrem_wsrem_stoprem,description_cont_lower_tagrem_puncrem_numrem_wsrem_stoprem_freqrem,description_cont_lower_tagrem_puncrem_numrem_wsrem_stoprem_freqrem_rarerem,description_cont_lower_tagrem_puncrem_numrem_wsrem_stoprem_freqrem_rarerem_taxo,description_cont_lower_tagrem_puncrem_numrem_wsrem_stoprem_freqrem_rarerem_taxo_stem
0,,22062762,When executing the Get Port Bandwidth command ...,When executing the Get Port Bandwidth command ...,when executing the get port bandwidth command ...,when executing the get port bandwidth command ...,when executing the get port bandwidth command ...,when executing the get port bandwidth command ...,when executing the get port bandwidth command ...,executing get port bandwidth command msl...,executing get port bandwidth command msle ssp ...,executing get port bandwidth command msle ssp ...,executing get port bandwidth command msle ssp ...,execut get port bandwidth command msle ssp tra...
1,,220152061,"<pre style=""word-wrap: break-word; white-space...","<pre style=""word-wrap: break-word; white-space...","<pre style=""word-wrap: break-word; white-space...","currently we are seeing 50dpm on this issue, w...",currently we are seeing 50dpm on this issue w...,currently we are seeing dpm on this issue we...,currently we are seeing dpm on this issue we h...,currently seeing dpm issue unit ...,currently seeing dpm issue unit usb port usb p...,currently seeing dpm issue unit usb port usb p...,currently seeing dpm issue unit usb port usb p...,current see dpm issu unit usb port usb port mi...
2,noise.cannot_reproduce,220421258,"<pre style=""word-wrap: break-word; white-space...","<pre style=""word-wrap: break-word; white-space...","<pre style=""word-wrap: break-word; white-space...",basin falls uses a server derived cpu with a k...,basin falls uses a server derived cpu with a k...,basin falls uses a server derived cpu with a k...,basin falls uses a server derived cpu with a k...,basin falls uses server derived cpu kbp ...,basin falls uses server derived cpu kbp pch de...,basin falls uses server derived cpu kbp pch de...,basin falls uses server derived cpu kbp pch de...,basin fall use server deriv cpu kbp pch design...
3,noise.cannot_reproduce,220634430,"<p style=""font-size: 12.18px;""><span style=""fo...","<p style=""font-size: 12.18px;""><span style=""fo...","<p style=""font-size: 12.18px;""><span style=""fo...",xhci_debug_device test failed on cnl b0 platfo...,xhci debug device test failed on cnl b0 platfo...,xhci debug device test failed on cnl b platfo...,xhci debug device test failed on cnl b platfor...,xhci debug device test failed cnl b platform...,xhci debug device test failed cnl platform fai...,xhci debug device test failed cnl platform fai...,xhci debug device failed cnl platform failin...,xhci debug devic fail cnl platform fail seed c...
4,noise.non-issue,220634437,<p>Trying Recipe sent by Tamir</p><p>pci_confi...,<p>Trying Recipe sent by Tamir</p><p>pci_confi...,<p>trying recipe sent by tamir</p><p>pci_confi...,trying recipe sent by tamir pci_config_registe...,trying recipe sent by tamir pci config registe...,trying recipe sent by tamir pci config registe...,trying recipe sent by tamir pci config registe...,trying recipe sent tamir pci config register...,trying recipe sent tamir pci config registers ...,trying recipe sent tamir pci config registers ...,trying recipe sent tamir pci config registers ...,tri recip sent tamir pci config regist access ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2073,,14010346691,"<div style=""direction:ltr"">\n\n<table border=""...","<div style=""direction:ltr""> <table border=""1"" ...","<div style=""direction:ltr""> <table border=""1"" ...",hardware asus prime z390-a haps80 

Target distribution: noise.validation_tools       249
noise.duplicated_sighting    178
bug.logic.xhci               164
noise.test_content           126
noise.non-issue              112
                            ... 
bug.logic.modphy               1
environment.board.crb          1
bug.logic.exi                  1
bug.logic.usb3                 1
bug.logic.pmc                  1
Name: problem_area, Length: 70, dtype: int64
Train-test split completed with 70.0 - 30.0 split in train-test
Shape of X_train is: (1454, 1)
Shape of X_test is: (624, 1)
Shape of y_train is: (1454,)
Shape of y_test is: (624,)
Shape of X_train after feature extraction: (1454, 246096)
Shape of X_test after feature extraction: (624, 246096)
Model saved!
Overall accuracy achieved is 19.23%
Classification report:
                                            precision    recall  f1-score   support

                                                0.19      0.12      0.15        33
                  bug.circuit.modphy-

C:\Users\nchong\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### Interface

In [7]:
#read config file and call the other functions
def main(config_file):
    
    import json     
    import pandas as pd  
    
    
    #---------DATA LOADING----------#           
    with open(config_file) as f:
        data = json.load(f)
        
    user_outpath = data["user_outpath"]    
    log_path = data["log_path"]
        
    dl = data["DataLoading"]
    path,start_date,stop_date = dl['path'],dl['start_date'],dl['stop_date']  
    df = data_loading(path=path,start_date=start_date,stop_date=stop_date)
    
    #---------DATA PREPROCESSING----------# 
    #df_manipulation
    dm = data["DataPreprocessing"]["df_manipulation"]
    how,col_selection,keep,subset = dm['how'],dm['col_selection'],dm['keep'],dm['subset']  
    df = df_manipulation(df,how=how,col_selection=col_selection,keep=keep,subset=subset)
    display(df)
    df_all = df.copy() #id, raw text -> data preprocessing final file
    df_out = df.copy() #id, raw text -> ml output
    df = df.drop("id",axis=1) #raw text for data preprocessing
    
    #remove target from df for supervised    
    if data["DataPreprocessing"]["target"]["enable"]:
        target = data["DataPreprocessing"]["target"]["column"]
        target = df[[target]] 
        df = df.drop(target,axis=1)
        df_all = df_all.drop(target,axis=1)
        display(df)
        display(target)
        
#    word_contractions
    wordcont = data["DataPreprocessing"]["word_contractions"]["enable"]    
    if wordcont:
        df = word_contractions(df)
        df_all = pd.concat([df_all,df],axis=1) #raw text, raw text after contractions
            
    #lowercase
    lower = data["DataPreprocessing"]["lowercase"]["enable"]      
    if lower:
        df = lowercase(df)
        df_all = pd.concat([df_all,df],axis=1)

            
   #Remove html tag and url
    tagrem = data["DataPreprocessing"]["remove_htmltag_url"]["enable"] 
    if tagrem:
        df = remove_htmltag_url(df)
        df_all = pd.concat([df_all,df],axis=1)

    #Remove irrelevant characters and punctuation
    pc = data["DataPreprocessing"]["remove_irrchar_punc"]
    puncrem,char = pc["enable"],pc["char"] 
    if puncrem:
        df = remove_irrchar_punc(df,char=char)
        df_all = pd.concat([df_all,df],axis=1)
        
    #Remove numbers 
    numrem = data["DataPreprocessing"]["remove_num"]["enable"]
    if numrem:
        df = remove_num(df)
        df_all = pd.concat([df_all,df],axis=1)

    # Remove multiple whitespace
    wsrem = data["DataPreprocessing"]["remove_multwhitespace"]["enable"]
    if wsrem:
        df = remove_multwhitespace(df)
        df_all = pd.concat([df_all,df],axis=1)

    # Remove stopwords
    sw = data["DataPreprocessing"]["remove_stopwords"]
    stoprem,extra_sw,remove_sw = sw["enable"],sw["extra_sw"],sw["remove_sw"]
    if stoprem:
        df = remove_stopwords(df,extra_sw=extra_sw,remove_sw=remove_sw)
        df_all = pd.concat([df_all,df],axis=1)
        
    # Remove frequent words
    fw = data["DataPreprocessing"]["remove_freqwords"]
    freqrem,n = fw["enable"],fw["n"]
    if freqrem:
        df = remove_freqwords(df,n)
        df_all = pd.concat([df_all,df],axis=1)
    
    # Remove rare words
    rw = data["DataPreprocessing"]["remove_rarewords"]
    rarerem,n = rw["enable"],rw["n"]    
    if rarerem:
        df = remove_rarewords(df,n)
        df_all = pd.concat([df_all,df],axis=1)
        
    # Custom taxonomy
    ct = data["DataPreprocessing"]["custom_taxo"]
    taxo,remove_taxo,include_taxo = ct["enable"], ct["remove_taxo"], ct["include_taxo"]
    if taxo:
        df = custom_taxo(df,remove_taxo,include_taxo)
        df_all = pd.concat([df_all,df],axis=1)     
        
    # Stemming
    st = data["DataPreprocessing"]["stem_words"]
    stem,stemmer_type = st["enable"],st["stemmer_type"]     
    if stem:
        df = stem_words(df,stemmer_type)
        df_all = pd.concat([df_all,df],axis=1)            
    
    #Lemmatization
    lem = data["DataPreprocessing"]["lemmatize_words"]
    lemma,lemma_type = lem["enable"], lem["lemma_type"]      
    if lemma:
        df = lemmatize_words(df,lemma_type)
        df_all = pd.concat([df_all,df],axis=1)
    
    #column bind target to df
    if data["DataPreprocessing"]["target"]["enable"]:
        df = pd.concat([target,df],axis=1)
        df_all = pd.concat([target,df_all],axis=1)
    
    display(df)
    display(df_all)
    
    
    df_all.to_csv(user_outpath+"preprocessed_text.csv",index=False) #save after data preprocessing   
    
    #---------------ML module---------------# 
    elk_outpath = data["UnsupervisedLearning"]["elk_outpath"]
   
    ####---Unsupervised Learning---###
    #check output save in user or ELK folder
    if data["UnsupervisedLearning"]["Output"]["User"]:#store output in user folder 
        outpath = user_outpath
    if data["UnsupervisedLearning"]["Output"]["ELK"]: #store output in ELK folder
        outpath = elk_outpath  

    #k-means clustering 
    km= data["UnsupervisedLearning"]["kmeans_clustering"]
    kmeans = km["enable"]
    if kmeans:
        top_n_terms,ngram_range,fe_type,n_clusters,max_n_clusters= km["top_n_terms"],km["ngram_range"],km["fe_type"],km["n_clusters"],km["max_n_clusters"]        
        df_out["cluster"]=kmeans_clustering(column=df,outpath=outpath,top_n_terms=top_n_terms,ngram_range=ngram_range,fe_type=fe_type,n_clusters=n_clusters,max_n_clusters=max_n_clusters)
        display(df_out)
        df_out.to_csv(outpath+"KMeansClustering_output.csv",index=False)          

    #LDA
    lda_m= data["UnsupervisedLearning"]["lda"]
    LatentDirichletAllocation = lda_m["enable"]
    if LatentDirichletAllocation:
        n_components,top_n_terms,ngram_range= lda_m["n_components"],lda_m["top_n_terms"],lda_m["ngram_range"]       
        df_out["cluster"]=lda(column=df,outpath=outpath,n_components=n_components,top_n_terms=top_n_terms,ngram_range=ngram_range)
        display(df_out)        
        df_out.to_csv(outpath+"LatentDirichletAllocation_output.csv",index=False)       

    #NMF Factorization
    nmf_m= data["UnsupervisedLearning"]["nmf"]
    NonNegativeMatrixFactorization = nmf_m["enable"]
    if NonNegativeMatrixFactorization:
        n_components,top_n_terms,fe_type,ngram_range= nmf_m["n_components"],nmf_m["top_n_terms"],nmf_m["fe_type"],nmf_m["ngram_range"]
        df_out["cluster"]=nmf(column=df,outpath=outpath,n_components=n_components,top_n_terms=top_n_terms,fe_type=fe_type,ngram_range=ngram_range)
        display(df_out)        
        df_out.to_csv(outpath+"NonNegativeMatrixFactorization_output.csv",index=False) 

      
    #####---Similarity Metrics-----####
    #Cosine Similarity
    cs= data["SimilarityMetrics"]["cosinesimilarity"]
    cosinesim = cs["enable"]
    if cosinesim:
        threshold,total_rows,base_row,ngram_range,fe_type,ascending= cs["threshold"],cs["total_rows"],cs["base_row"],cs["ngram_range"],cs["fe_type"],cs["ascending"]        
        cosinesimilarity(column=df,user_outpath=user_outpath,threshold=threshold,total_rows=total_rows,base_row=base_row,ngram_range=ngram_range,fe_type=fe_type,ascending=ascending)

    #Jaccard Similarity
    js= data["SimilarityMetrics"]["jaccardsimilarity"]
    jaccardsim = js["enable"]
    if jaccardsim:
        threshold,total_rows,base_row,ascending= js["threshold"],js["total_rows"],js["base_row"],js["ascending"]
        jaccardsimilarity(column=df,user_outpath=user_outpath,threshold=threshold,total_rows=total_rows,base_row=base_row,ascending=ascending)

    #####-----Supervised Learning----####
      
         
    #Machine Learning
    sl= data["SupervisedLearning"]["supervised_lng"]
    SupervisedLearning = sl["enable"]
    if SupervisedLearning:
        target,test_size,ngram_range,fe_type,model_type,ascend= sl["target"],sl["test_size"],sl["ngram_range"],sl["fe_type"],sl["model_type"],sl["ascend"]
        supervised_lng(df=df,user_outpath=user_outpath,target=target,test_size=test_size,ngram_range=ngram_range,fe_type=fe_type,model_type=model_type,ascend=ascend)

    #Deep Learning
    dl= data["SupervisedLearning"]["deep_lng"]
    DeepLearning = dl["enable"]
    if DeepLearning:
        target,test_size,ngram_range,fe_type,hidden_layer_sizes,activation,solver,learning_rate,max_iter,ascend= dl["target"],dl["test_size"],dl["ngram_range"],dl["fe_type"],dl["hidden_layer_sizes"],dl["activation"],dl["solver"],dl["learning_rate"],dl["max_iter"],dl["ascend"]
        deep_lng(df=df,user_outpath=user_outpath,target=target,test_size=test_size,ngram_range=ngram_range,fe_type=fe_type,hidden_layer_sizes=hidden_layer_sizes,activation=activation,solver=solver,learning_rate=learning_rate,max_iter=max_iter,ascend=ascend)



### Functions

In [8]:
#data loading
def data_loading(path,start_date=None,stop_date=None):
    '''
    Load only files that follow agreed filename format, merge files as single dataframe.
    User can choose to 
    a) Load all json files following the agreed filename format
    b) Load only json files from specific dates by adding the start and stop dates (Note: Both start_date and
    stop_date must be used together)
    
    params:
    path [string]: path of the files, without filename
    
    start_date[None/string in YYYY-MM-DD format](optional,default is None): 
    User can choose to load files starting from start_date
    - None: no start_date is provided, all files are loaded
    - string in YYYY-MM-DD format: files starting from start_date will be loaded
    
    stop_date[None/string in YYYY-MM-DD format](optional,default is None): 
    User can choose to load files until stop_date
    - None: no stop_date is provided, all files are loaded
    - string in YYYY-MM-DD format: files until stop_date will be loaded
    '''
    from datetime import datetime,timedelta
    import pandas as pd
    import glob, os, json
    import re
    
    filenames = os.listdir(path)
    file_list=[]
    date_list = []
    df = pd.DataFrame()
    
    if start_date == None and stop_date == None :
        for file in filenames:
            # search agreed file format pattern in the filename

            pattern = r"^\(\d{4}-\d{2}-\d{1,2}\)\d+\_\D+\_\d+\.json$"

            match = re.search(pattern,file)
                
            #if match is found
            if match:
                pattern = os.path.join(path, file) #join path with file name
                file_list.append(pattern) #list of json files that follow the agreed filename
            
        print("Files read:",file_list)                   
        for file in file_list:
            with open(file) as f:
                #flatten json into pd dataframe
                json_data = pd.json_normalize(json.loads(f.read()))
                json_data = pd.DataFrame(json_data)
                #label which file each row is from 
                json_data['file'] = file.rsplit("/", 1)[-1]

            df = df.append(json_data)              
                
    else:
        #convert start and stop string to datetime
        start = datetime.strptime(start_date, "%Y-%m-%d").date()
        stop = datetime.strptime(stop_date, "%Y-%m-%d").date()
    
        #iterate from start to stop dates by day and store dates in list
        while start <= stop:
            date_list.append(start)
            start = start + timedelta(days=1)  # increase day one by one

        #convert datetime objects to string
        string_list =[d.strftime("%Y-%m-%d") for d in date_list]
#         print(string_list)
        
        for file in filenames: 
            
            # search agreed file format pattern in the filename
            for date in string_list: 
                pattern = r"\("+date+r"\)\d+\_\D+\_\d+\.json"
        
                match = re.search(pattern,file)
                
                #if match is found
                if match:
                    pattern = os.path.join(path, file) #join path with file name
                    file_list.append(pattern) #list of json files that follow the agreed filename

        print("Files read:",file_list)     
        for file in file_list:
            with open(file) as f:
                #flatten json into pd dataframe
                json_data = pd.json_normalize(json.loads(f.read()))
                json_data = pd.DataFrame(json_data)
                #label which file each row is from 
                json_data['file'] = file.rsplit("/", 1)[-1]

            df = df.append(json_data)

    return df

In [2]:
#data preprocessing
def df_manipulation(df,how=None,col_selection=None,keep=None,subset=None):
    """
    1) Column selection: Keep columns in dataframe
    2) Data impute: Impute NA rows with empty string
    3) Data duplication cleaning: Drop all duplicates or drop all duplicates except for the first/last occurrence
    
    params:
    df [dataframe]: input dataframe  
    how[None/string]: Drop rows when we have at least one NA or all NA. Choose
                      # - None : NA imputed with empty string
                      # - "all": Drop row with all NA
                      # - "any": Drop row with at least one NA
    col_selection [None/list]: - None [Default]: Keep all columns in dataframe 
                               - List: List of columns to keep in dataframe                      
                                 
    keep[None/string/False]: Choose to drop all duplicates or drop all duplicates except for the first/last occurrence
                      # - None[DEFAULT] : Drop duplicates except for the first occurrence. 
                      # - "last" : Drop duplicates except for the last occurrence. 
                      # - False : Drop all duplicates.                 
    subset[list/None]: Subset of columns for dropping NA and identifying duplicates, use None if no column to select
   
    """
    
    print("Shape of df before manipulation:",df.shape)

    #Column selection - user can select column(s) 
    if col_selection != None:
        df = df[col_selection]
    
    print("Shape of df after selecting columns:",df.shape)

    #---Data impute - user can impute or drop rows with NA,freq of null values before & after manipulation returned---#
    print("Number of null values in df:\n",df.isnull().sum())         
    
    if how == None: # impute NA values with empty string
        print("NA is imputed with empty string")
        impute_value = ""
        df = df.fillna(impute_value)
        print("Number of null values in df after NA imputation:\n",df.isnull().sum())        
    else: # drop rows with NA values
        print("NA is dropped")
        df= df.dropna(axis=0, how=how,subset=subset)
        print("Number of null values in df after dropping NA rows:\n",df.isnull().sum())
        print("Shape of df after dropping NA rows:",df.shape)


    #---------Data duplication cleaning--------#
    print("Number of duplicates in the df:", df.duplicated().sum())

    #drop duplicates
    if keep==None:
        keep="first"
    df = df.drop_duplicates(subset=subset, keep=keep)

    print("Shape of df after manipulation:",df.shape)

    return df

def word_contractions(df):
    """
    Expand word contractions (i.e. "isn't" to "is not")
    params:
    df [dataframe]: input dataframe 
    """
    import contractions
    import pandas as pd
    df = df.applymap(lambda text: " ".join([contractions.fix(word) for word in text.split()]))
    df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
    
    df = df.add_suffix('_cont')
    
    return df

def lowercase(df):
    """
    Convert all characters to lower case
    param:
    df[dataframe]: input dataframe
    
    """
    import pandas as pd
    df = df.applymap(lambda s:s.lower() if type(s) == str else s)
    df = df.add_suffix('_lower')
        
    return df 

def remove_htmltag_url(df):
    """
    Remove html tag and url
    params:
    df [dataframe]: input dataframe 
    
    """
    from bs4 import BeautifulSoup
    #remove html tag
    df = df.applymap(lambda text:BeautifulSoup(text, 'html.parser').get_text(separator= " ",strip=True))
    #remove url
    df = df.replace('https?[://%]*\S+',' ', regex=True) 
    
    df = df.add_suffix('_tagrem')
    
    return df

def remove_irrchar_punc(df,char=None):
    """
    Remove irrelevant characters and punctuation. Optional: User can specify special characters to be removed in regex
    format.    
    params:    
    df [dataframe]: input dataframe 
    characters[string]: input regex of characters to be removed  
    
    """
    import re 
    if char != None:
        #Remove special characters given by user
        df = df.replace(char,' ',regex = True)
            
    # Remove utf-8 literals (i.e. \\xe2\\x80\\x8)
    df = df.replace(r'\\+x[\d\D][\d\D]',' ',regex = True)
        
    #Remove special characters and punctuation
    df = df.replace('[^\w\s]',' ',regex = True)
    df = df.replace(r'_',' ',regex = True)
    
    df = df.add_suffix('_puncrem')
    
    return df

def remove_num(df):
    """
    Remove numeric data
    params:
    df [dataframe]: input dataframe 
    
    """
    df=df.replace('\d+',' ', regex=True) 
    df = df.add_suffix('_numrem')
    
    return df 

def remove_multwhitespace(df):
    """
    Remove multiple white spaces
    params:
    df [dataframe]: input dataframe 
    
    """
    df = df.replace(' +',' ', regex=True)
    df = df.add_suffix('_wsrem')
    
    return df


def remove_stopwords(df,extra_sw=None,remove_sw=None):
    """
    Removes English stopwords. Optional: user can add own stopwords or remove words from English stopwords  
    params:
    df [dataframe]: input dataframe 
    extra_sw [list] (optional): list of words/phrase to be added to the stop words 
    remove_sw [list] (optional): list of words to be removed from the stop words 
    """
    import nltk
    from nltk.corpus import stopwords

    all_stopwords = stopwords.words('english')
    
    #default list of stopwords
    if extra_sw == None and remove_sw==None:
        all_stopwords = all_stopwords
        
    # add more stopwords
    elif remove_sw == None:
        all_stopwords.extend(extra_sw) #add to existing stop words list
        
    # remove stopwords from existing sw list
    elif extra_sw == None:
        all_stopwords = [e for e in all_stopwords if e not in remove_sw] #remove from existing stop words list
        
    # remove and add stopwords to existing sw list
    else:
        all_stopwords.extend(extra_sw) #add to existing stop words list
        all_stopwords = [e for e in all_stopwords if e not in remove_sw] #remove from existing stop words list
           
    for w in all_stopwords:
        pattern = r'\b'+w+r'\b'
        df = df.replace(pattern,' ', regex=True)
    
    df = df.add_suffix('_stoprem')
                   
    return df 

def remove_freqwords(df,n):
    """
    Remove n frequent words
    params:
    df [dataframe]: input dataframe 
    n [integer]: input number of frequent words to be removed
    """
    from collections import Counter
    cnt = Counter()
    for i in df:
    
        for text in df[i].values:
            for word in text.split():
                cnt[word] += 1
           
    #custom function to remove the frequent words             
    FREQWORDS = set([w for (w, wc) in cnt.most_common(n)])
    
    print("Frequent words that are removed:", set([(w, wc) for (w, wc) in cnt.most_common(n)]))
    df = df.applymap(lambda text: " ".join([word for word in str(text).split() if word not in FREQWORDS]))
    df = df.add_suffix('_freqrem')
    return df

def remove_rarewords(df,n):
    """
    Remove n rare words
    params:
    df [dataframe]: input dataframe 
    n [integer]: input number of rare words to be removed
    """
    from collections import Counter
    cnt = Counter()
    for i in df:
    
        for text in df[i].values:
            for word in text.split():
                cnt[word] += 1
           
    #custom function to remove the frequent words             
    RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n-1:-1]])
    
    print("Rare words that are removed:", set([(w,wc) for (w, wc) in cnt.most_common()[:-n-1:-1]]))
    df = df.applymap(lambda text: " ".join([word for word in str(text).split() if word not in RAREWORDS]))
    df = df.add_suffix('_rarerem')
    return df


def custom_taxo(df,remove_taxo,include_taxo):
    """
    User provides taxonomy to be removed or remained in the text. 
    a) user wants to remove taxonomies only -> input a list of taxonomies/regex to be removed in remove_taxo 
    b) user wants to remove taxonomies but wants the same taxonomy to remain in certain phrases 
    (i.e remove taxo "test" but  "test" remains in "test cycle") -> input a list of taxonomies to be removed in remove_taxo and list of
    phrases for the taxonomy to remain in include_taxo
    
    params:
    df [dataframe]: input dataframe
    remove_taxo[list/regex]: list of taxonomy to be removed from text
    include_taxo[list/None]: list of taxonomy to be maintained in text
    """
    import re
    import pandas as pd 
    
    def convert(text,remove_taxo):  
        """
        Uses regex given in remove_taxo to find and return all matches 
        """
        match = re.findall(remove_taxo,text)
        if match:                 
            new_row = {'Match':match}
            return(new_row)
        
    #if remove_taxo is regex call convert function to get all matches as a list
    if type(remove_taxo) == str: 
        cv_list = []
        for i in range(len(df.columns)):
            for text in df.iloc[:,i]:
                cv = convert(text,remove_taxo)
                if cv:
                    cv_list.append(cv)
        #             print(cv_list)

        cv_df = pd.DataFrame(cv_list)
        remove_taxo = list(cv_df["Match"].apply(pd.Series).stack().unique())
        print("Remove_taxo_list:", remove_taxo)
        
    def taxo(text,remove_taxo,include_taxo): 
        if remove_taxo != None and include_taxo != None: #user wants to remove taxonomies but wants the same taxonomy to remain in certain phrases (i.e remove "test" but remain "test" in "test cyccle")

            for w in remove_taxo:
            #row without any item from include_taxo -> replace all remove_taxo items with empty string
                if all(phrase not in text for phrase in include_taxo): 
                    pattern = r'\b'+w+r'\b'
                    text = re.sub(pattern,' ', text) 
                #row with any item from include_taxo -> only replace remove_taxo item that is not in include_taxo
                else: 
                    if all(w not in phrase for phrase in include_taxo):
                        pattern = r'\b'+w+r'\b'
                        text = re.sub(pattern,' ', text) 
                        
        if remove_taxo != None and include_taxo == None: #user wants to remove taxonomies only:
            for w in remove_taxo: #remove_taxo in list of words
                pattern = r'\b'+w+r'\b'
                text = re.sub(pattern,' ', text)
                 
        return text 
    
    
    df = df.applymap(lambda text: taxo(text,remove_taxo,include_taxo))     
    df = df.add_suffix('_taxo')
                
    return df    

def stem_words(df,stemmer_type):
    """
    Stemming words. Default option is Porter Stemmer, alternative option is Lancaster Stemmer 
    params:
    df[dataframe]: input dataframe
    stemmer_type[None/string]: input stemming method 
                                - None for Porter Stemmer
                                - "Lancaster" for Lancaster Stemmer 
    """
    import pandas as pd
    import nltk
    from nltk.stem import PorterStemmer
    from nltk.stem import LancasterStemmer
    
    if stemmer_type == None:
        stemmer = PorterStemmer()
    if stemmer_type == "Lancaster":
        stemmer=LancasterStemmer()
    df = df.applymap(lambda text: " ".join([stemmer.stem(word) for word in text.split()]))
    df = df.add_suffix('_stem')
    
    
    return df

def lemmatize_words(df,lemma_type):
    """
    Lemmatize words: Default option is WordNetLemmatizer, alternative option is Spacy 
    params:
    df[dataframe]: input dataframe
    lemma_type[None/string]: input lemmatization method
                            - None for WordNetLemmatizer
                            - "Spacy" for Spacy    
    """
    import pandas as pd
    import spacy
    import nltk
    from nltk.stem import WordNetLemmatizer
    
    if lemma_type == None:
        lemmatizer = WordNetLemmatizer()
        df = df.applymap(lambda text: " ".join([lemmatizer.lemmatize(word) for word in text.split()]))
        
    if lemma_type == "Spacy":
        nlp = spacy.load("en_core_web_sm")
        df = df.applymap(lambda text: " ".join([word.lemma_ for word in nlp(text)]))
        #convert to lower case as spacy will convert pronouns to upper case
        df = df.applymap(lambda s:s.lower() if type(s) == str else s) 
        
    df = df.add_suffix('_lemma')
        
    return df

def feature_extraction(column,ngram_range=None,ascending=None,fe_type=None):
    """
    Feature extraction methods - TF-IDF(default choice) or Bag of words
     
    params:
    column [series/DataFrame]: column selected for feature extraction 
                        - series: only one column is selected for feature extraction (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for feature extraction (e.g. df[["title_clean","desc_clean"]])
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       - [default] ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    ascending [True/False/None]: - [default] None (words arranged in alphabetical order)
                                 - True(words arranged in ascending order of sum), 
                                 - False(words arranged in descending order of sum)                               
    fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
    
    """
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    if type(column) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        column = column.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
                    
    if ngram_range == None: #set ngram range as unigram by default
        ngram_range=(1,1)
        
    if fe_type == "bagofwords":
        vec_type = CountVectorizer(ngram_range=ngram_range, analyzer='word')
        vectorized = vec_type.fit_transform(column)
        df = pd.DataFrame(vectorized.toarray(), columns=vec_type.get_feature_names())
        df.loc['sum'] = df.sum(axis=0).astype(int)

    if fe_type == None: #tfidf
        vec_type = TfidfVectorizer(ngram_range=ngram_range, analyzer='word')
        vectorized = vec_type.fit_transform(column)
        df = pd.DataFrame(vectorized.toarray(), columns=vec_type.get_feature_names())
        df.loc['sum'] = df.sum(axis=0)
    
    if ascending != None:
            
        df = df.sort_values(by ='sum', axis = 1,ascending=ascending)
    
    
    return df,vec_type,vectorized

In [10]:
#unsupervised learning
import pandas as pd
def kmeans_clustering(column,outpath,top_n_terms,ngram_range=None,fe_type=None,n_clusters=None,max_n_clusters=None):
    """
    K- means clustering for unsupervised learning. User can choose either options:
    (1) provide the number of clusters or
    (2) provide the max number of clusters for kmeans to iterate through, the optimal number of clusters with highest 
    silhouette score will be chosen. Min number of clusters is fixed as 2
    
    params:
    column [series/DataFrame]: column(s) selected for clustering 
                        - series: only one column is selected for clustering (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for clustering (e.g. df[["title_clean","desc_clean"]])
    top_n_terms[int]: the top n terms in each cluster to be printed out
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   - [default] ngram_range of (1, 1) means only unigrams, 
                                   - ngram_range of (1, 2) means unigrams and bigrams, 
                                   - ngram_range of (2, 2) means only bigram
    fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
    n_clusters[None/int]: number of clusters. Choose None for option (2)  
    max_n_clusters[None/int]: max number of clusters. Choose None for option (1)  
    """   
    from sklearn.cluster import KMeans
    from sklearn import metrics

    silhouette_avg_list = []
    n_clusters_list = []
    dicts = {}
    
    #call feature extraction function    
    ascending = None 
    X = feature_extraction(column,ngram_range,ascending,fe_type)[0]
    X = X.drop(index='sum')
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]

    #user provides the number of clusters        
    if n_clusters != None:
        model = KMeans(n_clusters = n_clusters, random_state=42)
        model.fit_predict(X)
        labels = model.labels_

        silhouette_score = metrics.silhouette_score(X, labels,random_state=42)
        with open(outpath+'TopWords_SilhouetteScore_KMeans.txt','w') as f:
            print("Silhouette score for",n_clusters,"clusters is",round(silhouette_score,3),file=f)
        
            
    #user provides the maximum number of clusters 
    if max_n_clusters != None:
        for n_clusters in range(2,max_n_clusters+1): 

            model = KMeans(n_clusters = n_clusters, random_state=42)
            model.fit_predict(X)
            labels = model.labels_

            silhouette_avg = metrics.silhouette_score(X, labels,random_state=42)
#             print("For n_clusters =", n_clusters,"The silhouette_score is :", round(silhouette_avg,3))

            silhouette_avg_list.append(silhouette_avg)
            n_clusters_list.append(n_clusters)


        for i in range(len(n_clusters_list)):
            dicts[n_clusters_list[i]] = silhouette_avg_list[i]

        n_clusters_max = max(dicts,key=dicts.get)
        silhouette_avg_max = max(dicts.values())

        model = KMeans(n_clusters = n_clusters_max, random_state=42)
        model.fit_predict(X)
        labels = model.labels_
        n_clusters = n_clusters_max
        with open(outpath+'TopWords_SilhouetteScore_KMeans.txt','w') as f:
            print("\nThe optimal number of clusters selected is",n_clusters_max,"with silhouette_score of",round(silhouette_avg_max,3),"\n",file=f) 
    
    with open(outpath+'TopWords_SilhouetteScore_KMeans.txt','a') as f:
        print("Top",top_n_terms,"terms per cluster:",file=f)
        order_centroids = model.cluster_centers_.argsort()[:, ::-1] #sort by descending order
        terms = vec_type.get_feature_names()
        for i in range(n_clusters):
            print("Cluster",i,file=f)
            print(['%s' % terms[ind] for ind in order_centroids[i, :top_n_terms]],file=f) #top n terms in each cluster
            print("\n",file=f)
   
               
    return labels


def lda(column,outpath,n_components,top_n_terms,ngram_range=None):
    """
    LDA for unsupervised learning. Bag of words is selected for feature extraction
    params:
    column [series/DataFrame]: column(s) selected for lda
                        - series: only one column is selected for lda (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for lda (e.g. df[["title_clean","desc_clean"]])
    n_components[int]: the number of topics/clusters used in the lda_model
    top_n_terms[int]: the top n terms in each topic/cluster to be printed out
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   - [default] ngram_range of (1, 1) means only unigrams, 
                                   - ngram_range of (1, 2) means unigrams and bigrams, 
                                   - ngram_range of (2, 2) means only bigram
    
    """
    from sklearn.decomposition import LatentDirichletAllocation
    
    #feature extraction
    ascending = None
    fe_type = "bagofwords"
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    vectorized = feature_extraction(column,ngram_range,ascending,fe_type)[2]

    # Create object for the LDA class 
    lda_model = LatentDirichletAllocation(n_components, random_state = 42)  
    lda_model.fit(vectorized)
    
    # Components_ gives us our topic distribution 
    topic_words = lda_model.components_

    # Top n words for a topic
    with open(outpath+'TopWords_LDA.txt','w') as f:
        for i,topic in enumerate(topic_words):
            print(f"The top {top_n_terms} words for cluster #{i}",file=f)
            print([vec_type.get_feature_names()[index] for index in topic.argsort()[-top_n_terms:]],file=f)
            print("\n",file=f)
            
            
    topic_results = lda_model.transform(vectorized) #probabilities of doc belonging to particular topic
    
    
    return topic_results.argmax(axis=1)


def nmf(column,outpath,n_components,top_n_terms,fe_type,ngram_range=None):
    """
    Non-negative matrix factorization for unsupervised learning.
    params:
    column [series/DataFrame]: column(s) selected for NMF 
                        - series: only one column is selected for NMF (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for NMF (e.g. df[["title_clean","desc_clean"]])
    n_components[int]: the number of topics/clusters used in NMF
    top_n_terms[int]: the top n terms in each topic/cluster to be printed out
    fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   - [default] ngram_range of (1, 1) means only unigrams, 
                                   - ngram_range of (1, 2) means unigrams and bigrams, 
                                   - ngram_range of (2, 2) means only bigram
    """
    from sklearn.decomposition import NMF
    
    #feature extraction
    ngram_range = None
    ascending = None
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    vectorized = feature_extraction(column,ngram_range,ascending,fe_type)[2]

    # Create object for the NMF class 
    nmf_model = NMF(n_components,random_state=42)
    nmf_model.fit(vectorized)
    
    # Components_ gives us our topic distribution 
    topic_words = nmf_model.components_

    # Top n words for a topic
    with open(outpath+'TopWords_NMF.txt','w') as f:
        for i,topic in enumerate(topic_words):
            print(f"The top {top_n_terms} words for cluster #{i}",file=f)
            print([vec_type.get_feature_names()[index] for index in topic.argsort()[-top_n_terms:]],file=f)
            print("\n",file=f)
        
    topic_results = nmf_model.transform(vectorized) 
    
    return topic_results.argmax(axis=1)

In [11]:
#supervised learning

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import metrics
import joblib
import numpy as np 

def supervised_lng(df,user_outpath,target,test_size,ngram_range=None,fe_type=None,model_type=None,ascend=None):

    """
    Consists of 3 supervised machine learning methods: RandomForest (Default), Naive Bayes(optional, SVM (optional)
    
    X[series/DataFrame]: column(s) of text for supervised learning
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]])
    y[series]: target 
    test_size[float/int]: If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
                          If int, represents the absolute number of test samples.
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       -[DEFAULT] ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    fe_type[None/string]: Feature extraction type: Choose "bagofwords" or None for default tfidf method
    model_type[None/string]: Choose ML algorithm 
                            - None (Default algorithm is Random Forest)
                            - 'NB'(To choose Naive Bayes as ML algorithm), 
                            - 'SVM'(To choose Support Vector Machine as ML algorithm)
    ascend[True/False/None]:  - None (Default: Confusion matrix is arranged in alphabetical order)
                              - True(Confusion matrix arranged in ascending order of accuracy % per label), 
                              - False(Confusion matrix arranged in descending order of accuracy % per label)  
    save_path[None/string]: Path to save model
                            - None (Default - Model is not saved)
                            - String (Model is saved as model.joblib in the save_path specified as a string)
        
    """
    print("Target distribution:",df[target].value_counts())
    X= df.drop([target],axis=1)
    y= df[target]   
    
    #TRAIN-TEST SPLIT
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    print("Train-test split completed with",(1-test_size)*100,"-",test_size*100,"split in train-test")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_test is:",X_test.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_test is:",y_test.shape)
    
    if type(X_train) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_train = X_train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)         

    if type(X_test) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_test = X_test.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

    
    #FEATURE EXTRACTION
    column = X_train       
    ascending = None
    #fit_transform X_train
    X_train = feature_extraction(column,ngram_range,ascending,fe_type)[2]
    #only transform X_test
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    X_test = vec_type.transform(X_test)
    
    
    print("Shape of X_train after feature extraction:",X_train.shape)
    print("Shape of X_test after feature extraction:",X_test.shape)
    
    #MODEL BUILDING
    if model_type == None:
        #random forest is chosen by default
        model = RandomForestClassifier(random_state = 42)
    
    if model_type == "NB":
        model = MultinomialNB()
                   
    if model_type == "SVM":
        model = svm.SVC(random_state = 42)
    
    model.fit(X_train, y_train) 
    
    #MODEL SAVING
    
    joblib.dump(model, user_outpath + "model.joblib")
    print("Model saved!")

    # predicting test set results
    y_pred = model.predict(X_test)

    # MODEL EVALUATION  
   
    print('Overall accuracy achieved is ' + str(round(metrics.accuracy_score(y_test, y_pred)*100,2)) + "%")
    print("Classification report:\n",metrics.classification_report(y_test, y_pred,zero_division=0))
    
    #overall accuracy
    overall_acc = round(metrics.accuracy_score(y_test, y_pred)*100,2)
    overall_acc = {'Overall Acc %':overall_acc}
    overall_acc = pd.DataFrame([overall_acc])
    overall_acc.to_csv(user_outpath+"Overall_Accuracy.csv")

    #classification report
    report = metrics.classification_report(y_test, y_pred,zero_division=0,output_dict=True)
    report = pd.DataFrame(report).transpose()
    report.to_csv(user_outpath+"Classification_Report.csv")

    #confusion matrix with accuracies for each label
    class_accuracies = []

    for class_ in y_test.sort_values(ascending= True).unique():
        class_acc = round(np.mean(y_pred[y_test == class_] == class_)*100,2)
        class_accuracies.append(class_acc)
    class_acc = pd.DataFrame(class_accuracies,index=y_test.sort_values(ascending= True).unique(),columns= ["Accuracy %"])

    cf_matrix = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred, labels= y_test.sort_values(ascending= True).unique()), 
        index=y_test.sort_values(ascending= True).unique(), 
        columns=y_test.sort_values(ascending= True).unique()
    )
    
    if ascend == None:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
    else:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)
          
    cf_matrix.to_csv(user_outpath+"Confusion_Matrix.csv",index=False)  

#Deep Learning 
def deep_lng(df,user_outpath,target,test_size,ngram_range,fe_type,hidden_layer_sizes=None,activation=None,solver=None,learning_rate=None,max_iter=None,ascend=None):
    """
     Deep learning method: MultiLayer Perceptron

    X[series/DataFrame]: column(s) of text for deep learning
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]])   
    y[series]: target
    test_size[float/int]: If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
                          If int, represents the absolute number of test samples.
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       - ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    fe_type[string]: Feature extraction type: Choose "bagofwords" or "tfidf" method
    hidden_layer_sizes[tuple],default = (100): To set the number of layers and the number of nodes.
                                               Each element in the tuple represents the number of nodes,
                                               length of tuple denotes the total number of hidden layers in the network
    activation["identity", "logistic", "tanh","relu"], default="relu": Activation function for the hidden layer.
    solver["lbfgs", "sgd", "adam"], default="adam": The solver for weight optimization.
    learning_rate["constant", "invscaling", "adaptive"], default="constant": Learning rate schedule for weight updates
    max_iter[int], default=200: Maximum number of iterations. The solver iterates until convergence or this number of iterations.
    ascend [True/False/None]: - None (Default: Confusion matrix is arranged in alphabetical order)
                                 - True(Confusion matrix arranged in ascending order of accuracy % per label), 
                                 - False(Confusion matrix arranged in descending order of accuracy % per label)                            
    save_path[None/string]: Path to save model
                            - None (Default - Model is not saved)
                            - String (Model is saved as model.joblib in the save_path specified as a string)    
    """    
    print("Target distribution:",df[target].value_counts())
    X= df.drop([target],axis=1)
    y= df[target]   
    
    #train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    print("Train-test split completed with",(1-test_size)*100,"-",test_size*100,"split in train-test")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_test is:",X_test.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_test is:",y_test.shape)
    
    if type(X_train) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_train = X_train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)         
        
    if type(X_test) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_test = X_test.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        
    #FEATURE EXTRACTION
    column = X_train
    ascending = None
    #fit_transform X_train
    X_train = feature_extraction(column,ngram_range,ascending,fe_type)[2]
    #only transform X_test
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    X_test = vec_type.transform(X_test)
    print("Shape of X_train after feature extraction:",X_train.shape)
    print("Shape of X_test after feature extraction:",X_test.shape)
    
    #MODEL BUILDING
    #default hypermarameters
    if hidden_layer_sizes == None:
        hidden_layer_sizes = (100)
    if activation == None:
        activation = "relu"
    if solver == None:
        solver = "adam"
    if learning_rate == None:
        learning_rate = "constant"
    if max_iter == None:
        max_iter = 200
    
    print("Hidden layer sizes: ", hidden_layer_sizes,", Activation: ",activation,", Solver: ",solver,", Learning rate: ",learning_rate,", Max iteration: ",max_iter)
    
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter,verbose = False,random_state=42)
    model.fit(X_train,y_train)
    
    
    #MODEL SAVING    
    joblib.dump(model, user_outpath + "mlpmodel.joblib")
    print("Model saved!")

    # predicting test set results
    y_pred = model.predict(X_test)

    # MODEL EVALUATION  
   
    print('Overall accuracy achieved is ' + str(round(metrics.accuracy_score(y_test, y_pred)*100,2)) + "%")
    print("Classification report:\n",metrics.classification_report(y_test, y_pred,zero_division=0))
    
    #overall accuracy
    overall_acc = round(metrics.accuracy_score(y_test, y_pred)*100,2)
    overall_acc = {'Overall Acc %':overall_acc}
    overall_acc = pd.DataFrame([overall_acc])
    overall_acc.to_csv(user_outpath+"Overall_Accuracy.csv")

    #classification report
    report = metrics.classification_report(y_test, y_pred,zero_division=0,output_dict=True)
    report = pd.DataFrame(report).transpose()
    report.to_csv(user_outpath+"Classification_Report.csv")

    #confusion matrix with accuracies for each label
    class_accuracies = []

    for class_ in y_test.sort_values(ascending= True).unique():
        class_acc = round(np.mean(y_pred[y_test == class_] == class_)*100,2)
        class_accuracies.append(class_acc)
    class_acc = pd.DataFrame(class_accuracies,index=y_test.sort_values(ascending= True).unique(),columns= ["Accuracy %"])

    cf_matrix = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred, labels= y_test.sort_values(ascending= True).unique()), 
        index=y_test.sort_values(ascending= True).unique(), 
        columns=y_test.sort_values(ascending= True).unique()
    )
    
    if ascend == None:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
    else:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)
          
    cf_matrix.to_csv(user_outpath+"Confusion_Matrix.csv",index=False)   
    

In [17]:
#similarity metrics
#Cosine Similarity 

def cosinesimilarity(column,user_outpath,threshold=None,total_rows = None,base_row=None,ngram_range=None,fe_type=None,ascending=None):
    """
    Compute the cosine similarity between rows of texts. User can 
    a) fix number of rows for comparison, each row will be taken as base and compared with the rest
    b) fix one row as base, comparison will be done with all the other rows
    
    params:    
    column[series/DataFrame]: column(s) of text for row wise similarity comparison
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]])  
    threshold[None/float]: cut off value for the cosine similarity, only texts with values above or equal to threshold
                           will be printed
                        - None: Default threhold is 0.5
                        - float: any value between 0 and 1 
    total_rows[None/int]: Number of rows for comparison, choose None for option b 
    base_row[None/int]: Row fixed as base, choose None for option a 
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       - ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    fe_type[None/string]: Feature extraction type: Choose "bagofwords" or None for tfidf
    ascending [True/False/None]: - [default] None (words arranged in alphabetical order)
                                 - True(words arranged in ascending order of sum), 
                                 - False(words arranged in descending order of sum)  
    
    """
    from sklearn.metrics.pairwise import cosine_similarity
    
    if type(column) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        column = column.apply(lambda row: ' '.join(row.values.astype(str)), axis=1) 
                
    #feature extraction              
    X = feature_extraction(column=column,ngram_range=ngram_range,ascending=None,fe_type=fe_type)[0]
    X = X.drop(["sum"],axis = 0)
    
    #Get cosine similarity matrix
    similarity_matrix = pd.DataFrame(cosine_similarity(X))
    
    #threshold
    if threshold == None:
        threshold = 0.5
        
       
    if total_rows !=None: #fix number of rows for comparison, each row will be taken as base and compared with the rest
        print("Fix number of rows for comparison, each row will be taken as base and compared with the rest")
        results_append = []
        for base in range(total_rows):            
            #Create empty df
            column_names = ["Base Index","Index", "Similarity Score", "Text"]
            results = pd.DataFrame(columns = column_names)            
            
            for i in range(total_rows): #compare base with other index
                
                if similarity_matrix.iloc[base,i] >= threshold: #print if comparison shows that silarity metric is more than threshold
                    new_row = {'Base Index':base,'Index':i,'Similarity Score':round(similarity_matrix.iloc[base,i],4), 'Text':column.iloc[i]}
                    #append row to the dataframe
                    results = results.append(new_row, ignore_index=True)
                
                    if ascending != None:            
                        results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)
                        
#             display(results)
            results_append.append(results)
        results_append = pd.concat(results_append)
        display(results_append)         
        results_append.to_csv(user_outpath+"Cosine_Similarity.csv",index=False)
            
    if base_row !=None: #fix base_row index for comparison with all indexes
        print ("Fix one row as base, comparison will be done with all the other rows")          
        #Create empty df
        column_names = ["Base Index","Index", "Similarity Score", "Text"]
        results = pd.DataFrame(columns = column_names)
        
        for i in range(len(column)): #compare base_row with other index
            if similarity_matrix.iloc[base_row,i] >= threshold: #print if comparison shows that silarity metric is more than threshold
                new_row = {'Base Index':base_row,'Index':i, 'Similarity Score':round(similarity_matrix.iloc[base_row,i],4), 'Text':column.iloc[i]}
                #append row to the dataframe
                results = results.append(new_row, ignore_index=True)
                if ascending != None:            
                    results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)  
#         display(results)
                
        results.to_csv(user_outpath+"Cosine_Similarity.csv",index=False)  

#Jaccard Similarity 

def jaccardsimilarity(column,user_outpath,threshold=None,total_rows = None,base_row=None,ascending=None):
    """
    Compute the jaccard similarity between texts. User can 
    a) fix number of rows for comparison, each row will be taken as base and compared with the rest
    b) fix one row as base, comparison will be done with all the other rows
    
    params:
    column[series/DataFrame]: column(s) of text for row wise similarity comparison
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]]) 
    threshold[None/float]: cut off value for the jaccard similarity, only texts with values above or equal to threshold
                           will be printed
                        - None: Default threhold is 0.5
                        - float: any value between 0 and 1 
    total_rows[None/int]: Number of rows for comparison, choose None for option b 
    base_row[None/int]: Row fixed as base, choose None for option a 
    ascending [True/False/None]: - [default] None (words arranged in alphabetical order)
                                 - True(words arranged in ascending order of sum), 
                                 - False(words arranged in descending order of sum)  
    
    """     
            
    #jaccard score computation
    def get_jaccard_sim(str1, str2):        
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    
    if type(column) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        column = column.apply(lambda row: ' '.join(row.values.astype(str)), axis=1) 
       
    #threshold
    if threshold == None:
        threshold = 0.5
        
    
    if total_rows !=None: #fix number of rows for comparison, each row will be taken as base and compared with the rest
        print("Fix number of rows for comparison, each row will be taken as base and compared with the rest")
        results_append = []
        for base in range(total_rows):
            
            #Create empty df
            column_names = ["Base Index","Index", "Similarity Score", "Text"]
            results = pd.DataFrame(columns = column_names)                   
            
            for i in range(total_rows): #compare base with other index
                jac_score =  round(get_jaccard_sim(column.iloc[base],column.iloc[i]),4)
                if jac_score >= threshold: #print if comparison shows that silarity metric is more than threshold
                    new_row = {"Base Index":base,'Index':i, 'Similarity Score':jac_score, 'Text':column.iloc[i]}
                    #append row to the dataframe
                    results = results.append(new_row, ignore_index=True)
                
                if ascending != None:            
                    results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)  
                    
#             display(results)
            results_append.append(results)
        results_append = pd.concat(results_append)
        display(results_append)         
        results_append.to_csv(user_outpath+"Jaccard_Similarity.csv",index=False)
        
    if base_row != None: #fix base_row index for comparison with all indexes
       
        print ("Fix one row as base, comparison will be done with all the other rows") 
        #Create empty df
        column_names = ["Base Index","Index", "Similarity Score", "Text"]
        results = pd.DataFrame(columns = column_names)                   
            
        for i in range(len(column)): #compare base_row with other index
            jac_score = round(get_jaccard_sim(column.iloc[base_row],column.iloc[i]),4)
            if jac_score >= threshold: #print if comparison shows that silarity metric is more than threshold
                new_row = {"Base Index":base_row,'Index':i, 'Similarity Score':jac_score, 'Text':column.iloc[i]}
                #append row to the dataframe
                results = results.append(new_row, ignore_index=True)
            if ascending != None:            
                results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)  
#         display(results)
        results.to_csv(user_outpath+"Jaccard_Similarity.csv",index=False)  

In [ ]:
# #convert csv to json
# import csv 
# import json


# def csv_to_json(csvFilePath, jsonFilePath):
#     jsonArray = []
      
#     #read csv file
#     with open(csvFilePath) as csvf: 
#         #load csv file data using csv library's dictionary reader
#         csvReader = csv.DictReader(csvf) 

#         #convert each csv row into python dict
#         for row in csvReader: 
#             #add this python dict to json array
#             jsonArray.append(row)
  
#     #convert python jsonArray to JSON String and write to file
#     with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
#         jsonString = json.dumps(jsonArray, indent=4)
#         jsonf.write(jsonString)
          
# csvFilePath = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/VICE/python_ir/sip_sighting_usb_duplicate_ai.csv"
# jsonFilePath = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/VICE_JSON/(2021-08-25)1_VICE_1.json"

# csv_to_json(csvFilePath, jsonFilePath)


In [ ]:
# import pandas as pd
# path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/VICE/python_ir/"
# df= pd.read_csv(path+"sip_sighting_usb_duplicate_ai.csv")
# df.head()

In [ ]:
# #unsupervised learning
# import pandas as pd
# def kmeans_clustering(column,top_n_terms,ngram_range=None,fe_type=None,n_clusters=None,max_n_clusters=None):
#     """
#     K- means clustering for unsupervised learning. User can choose either options:
#     (1) provide the number of clusters or
#     (2) provide the max number of clusters for kmeans to iterate through, the optimal number of clusters with highest 
#     silhouette score will be chosen. Min number of clusters is fixed as 2
    
#     params:
#     column [series/DataFrame]: column(s) selected for clustering 
#                         - series: only one column is selected for clustering (e.g. df["title_clean"])
#                         - DataFrame: more than one column is selected for clustering (e.g. df[["title_clean","desc_clean"]])
#     top_n_terms[int]: the top n terms in each cluster to be printed out
#     ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
#                                    - [default] ngram_range of (1, 1) means only unigrams, 
#                                    - ngram_range of (1, 2) means unigrams and bigrams, 
#                                    - ngram_range of (2, 2) means only bigram
#     fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
#     n_clusters[None/int]: number of clusters. Choose None for option (2)  
#     max_n_clusters[None/int]: max number of clusters. Choose None for option (1)  
#     """   
#     from sklearn.cluster import KMeans
#     from sklearn import metrics

#     silhouette_avg_list = []
#     n_clusters_list = []
#     dicts = {}
    
#     #call feature extraction function    
#     ascending = None 
#     X = feature_extraction(column,ngram_range,ascending,fe_type)[0]
#     X = X.drop(index='sum')
#     vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]

#     #user provides the number of clusters        
#     if n_clusters != None:
#         model = KMeans(n_clusters = n_clusters, random_state=42)
#         model.fit_predict(X)
#         labels = model.labels_

#         silhouette_score = metrics.silhouette_score(X, labels,random_state=42)
#         print("Silhouette score for",n_clusters,"clusters is",round(silhouette_score,3))
        
            
#     #user provides the maximum number of clusters 
#     if max_n_clusters != None:
#         for n_clusters in range(2,max_n_clusters+1): 

#             model = KMeans(n_clusters = n_clusters, random_state=42)
#             model.fit_predict(X)
#             labels = model.labels_

#             silhouette_avg = metrics.silhouette_score(X, labels,random_state=42)
#             print("For n_clusters =", n_clusters,"The silhouette_score is :", round(silhouette_avg,3))

#             silhouette_avg_list.append(silhouette_avg)
#             n_clusters_list.append(n_clusters)


#         for i in range(len(n_clusters_list)):
#             dicts[n_clusters_list[i]] = silhouette_avg_list[i]

#         n_clusters_max = max(dicts,key=dicts.get)
#         silhouette_avg_max = max(dicts.values())

#         model = KMeans(n_clusters = n_clusters_max, random_state=42)
#         model.fit_predict(X)
#         labels = model.labels_
#         n_clusters = n_clusters_max
#         print("\nThe optimal number of clusters selected is",n_clusters_max,"with silhouette_score of",round(silhouette_avg_max,3),"\n") 
        
#     print("Top",top_n_terms,"terms per cluster:")
#     order_centroids = model.cluster_centers_.argsort()[:, ::-1] #sort by descending order
#     terms = vec_type.get_feature_names()
#     for i in range(n_clusters):
#         print("Cluster %d:" % i)
#         print(['%s' % terms[ind] for ind in order_centroids[i, :top_n_terms]]) #top n terms in each cluster
#         print("\n")
   
               
#     return labels


# def lda(column,n_components,top_n_terms,ngram_range=None):
#     """
#     LDA for unsupervised learning. Bag of words is selected for feature extraction
#     params:
#     column [series/DataFrame]: column(s) selected for lda
#                         - series: only one column is selected for lda (e.g. df["title_clean"])
#                         - DataFrame: more than one column is selected for lda (e.g. df[["title_clean","desc_clean"]])
#     n_components[int]: the number of topics/clusters used in the lda_model
#     top_n_terms[int]: the top n terms in each topic/cluster to be printed out
#     ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
#                                    - [default] ngram_range of (1, 1) means only unigrams, 
#                                    - ngram_range of (1, 2) means unigrams and bigrams, 
#                                    - ngram_range of (2, 2) means only bigram
    
#     """
#     from sklearn.decomposition import LatentDirichletAllocation
    
#     #feature extraction
#     ascending = None
#     fe_type = "bagofwords"
#     vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
#     vectorized = feature_extraction(column,ngram_range,ascending,fe_type)[2]

#     # Create object for the LDA class 
#     lda_model = LatentDirichletAllocation(n_components, random_state = 42)  
#     lda_model.fit(vectorized)
    
#     # Components_ gives us our topic distribution 
#     topic_words = lda_model.components_

#     # Top n words for a topic

#     for i,topic in enumerate(topic_words):
#         print(f"The top {top_n_terms} words for topic #{i}")
#         print([vec_type.get_feature_names()[index] for index in topic.argsort()[-top_n_terms:]])
#         print("\n")
        
#     topic_results = lda_model.transform(vectorized) #probabilities of doc belonging to particular topic
    
    
#     return topic_results.argmax(axis=1)


# def nmf(column,n_components,top_n_terms,fe_type,ngram_range=None):
#     """
#     Non-negative matrix factorization for unsupervised learning.
#     params:
#     column [series/DataFrame]: column(s) selected for NMF 
#                         - series: only one column is selected for NMF (e.g. df["title_clean"])
#                         - DataFrame: more than one column is selected for NMF (e.g. df[["title_clean","desc_clean"]])
#     n_components[int]: the number of topics/clusters used in NMF
#     top_n_terms[int]: the top n terms in each topic/cluster to be printed out
#     fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
#     ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
#                                    - [default] ngram_range of (1, 1) means only unigrams, 
#                                    - ngram_range of (1, 2) means unigrams and bigrams, 
#                                    - ngram_range of (2, 2) means only bigram
#     """
#     from sklearn.decomposition import NMF
    
#     #feature extraction
#     ngram_range = None
#     ascending = None
#     vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
#     vectorized = feature_extraction(column,ngram_range,ascending,fe_type)[2]

#     # Create object for the NMF class 
#     nmf_model = NMF(n_components,random_state=42)
#     nmf_model.fit(vectorized)
    
#     # Components_ gives us our topic distribution 
#     topic_words = nmf_model.components_

#     # Top n words for a topic

#     for i,topic in enumerate(topic_words):
#         print(f"The top {top_n_terms} words for topic #{i}")
#         print([vec_type.get_feature_names()[index] for index in topic.argsort()[-top_n_terms:]])
#         print("\n")
        
#     topic_results = nmf_model.transform(vectorized) 
    
#     return topic_results.argmax(axis=1)

In [ ]:
# #write config file for supervised
# import json
# json_path ='C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/ML_Testing/'
# path = json_path + "VICE_JSON/"
# # path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/"
# config = {   
#     "Email":"abc@intel.com",
#     "DataLoading": {"path": path, "start_date": None,"stop_date":None},
#     "DataPreprocessing":{    
#     "df_manipulation": {"how":"any","col_selection":["id","title","description","problem_area"],"keep":"first","subset":None},   
#     "target":{"column":"problem_area"}, #for supervised learning only
#     "word_contractions": {"enable": True},
#     "lowercase": {"enable": True},
#     "remove_htmltag_url": {"enable":True},
#     "remove_irrchar_punc":{"enable":True,"char":None},
#     "remove_num":{"enable":True},
#     "remove_multwhitespace":{"enable":True},
#     "remove_stopwords":{"enable":True,"extra_sw":None,"remove_sw": None},
#     "remove_freqwords":{"enable":True,"n":10},
#     "remove_rarewords":{"enable":True,"n":10},
#     "custom_taxo":{"enable":True,"remove_taxo":["gio","fields","test"],"include_taxo":["test suite execution","kpi metric"]},
#     "stem_words":{"enable":True,"stemmer_type":None},
#     "lemmatize_words":{"enable":False,"lemma_type":None}
#     },   
#     "SupervisedLearning":{
#         "supervised_lng":{"enable":False,"target":"problem_area","test_size":0.3,"ngram_range":None,"fe_type":None,"model_type":None,"ascend":None},
#         "deep_lng":{"enable":True,"target":"problem_area","test_size":0.3,"ngram_range":None,"fe_type":None,"hidden_layer_sizes":(5),"activation":None,"solver":None,"learning_rate":None,"max_iter":None,"ascend":None}
#     }

# }

# with open(json_path+'config_supervised.json', 'w') as f:
#     json.dump(config,f,indent=8)

In [ ]:
# def custom_taxo(df,remove_taxo,include_taxo):
#     """
#     User provides taxonomy to be removed or remained in the text. 
#     a) user wants to remove taxonomies only -> input a list of taxonomies to be removed in remove_taxo 
#     b) user wants to remove taxonomies but wants the same taxonomy to remain in certain phrases 
#     (i.e remove taxo "test" but  "test" remains in "test cycle") -> input a list of taxonomies to be removed in remove_taxo and list of
#     phrases for the taxonomy to remain in include_taxo
    
#     params:
#     df [dataframe]: input dataframe
#     remove_taxo[list]: list of taxonomy to be removed from text
#     include_taxo[list/None]: list of taxonomy to be maintained in text
#     """
#     import re
    
#     def taxo(text,remove_taxo,include_taxo):
#         if remove_taxo != None and include_taxo != None: #user wants to remove taxonomies but wants the same taxonomy to remain in certain phrases (i.e remove "test" but remain "test" in "test cyccle")
#             for w in remove_taxo:
#             #row without any item from include_taxo -> replace all remove_taxo items with empty string
#                 if all(phrase not in text for phrase in include_taxo): 
#                     pattern = r'\b'+w+r'\b'
#                     text = re.sub(pattern,' ', text) 
#                 #row with any item from include_taxo -> only replace remove_taxo item that is not in include_taxo
#                 else: 
#                     if all(w not in phrase for phrase in include_taxo):
#                         pattern = r'\b'+w+r'\b'
#                         text = re.sub(pattern,' ', text) 
                        
#         if remove_taxo != None and include_taxo == None: #user wants to remove taxonomies only:
#              for w in remove_taxo:
#                 pattern = r'\b'+w+r'\b'
#                 text = re.sub(pattern,' ', text)
                 
#         return text 
    
    
#     df = df.applymap(lambda text: taxo(text,remove_taxo,include_taxo))    
#     df = df.add_suffix('_taxo')
                
#     return df    
